In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\jeand\AppData\Local\Temp\ipykernel_22244\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
# Your code

from sklearn.model_selection import train_test_split

X = data.iloc[:, 0]  # The SMS text messages
y = data.iloc[:, 1]  # The labels (SPAM or HAM)

# Split 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))

Training set size: 800
Test set size: 200


## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
# Your code

import re

def clean_html(text):
    # Remove inline JavaScript/CSS: <script>...</script> or <style>...</style>
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)

    # Remove HTML comments <!-- ... -->
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    return text

# Ensure all elements are treated as strings before cleaning
X_train = X_train.astype(str).apply(clean_html)
X_test = X_test.astype(str).apply(clean_html)




- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
# Your code

def clean_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove all single characters (regardless of position)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove prefixed "b" (from byte strings, if any)
    text = re.sub(r'^b\s+', '', text)
    
    # Final trim
    return text.strip()

X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)


## Now let's work on removing stopwords
Remove the stopwords.

In [8]:
# Your code

import nltk
from nltk.corpus import stopwords

# Download stopwords if not already available
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

X_train = X_train.apply(remove_stopwords)
X_test = X_test.apply(remove_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [9]:
# Your code

import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet data if needed
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized)

X_train = X_train.apply(lemmatize_text)
X_test = X_test.apply(lemmatize_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jeand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jeand\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [10]:
# Your code

import pandas as pd

# Combine the corrected X_train and y_train
df_train = pd.DataFrame({'text': X_train, 'label': y_train})

# Function to get top N words for a given label (0=ham, 1=spam)
def get_top_words(df, label, n=10):
    texts = df[df['label'] == label]['text']
    all_words = ' '.join(texts).split()
    word_freq = pd.Series(all_words).value_counts()
    return word_freq.head(n)

# Top 10 for HAM (label 0)
top_ham_words = get_top_words(df_train, label=0)
print("Top 10 HAM words:\n", top_ham_words)

# Top 10 for SPAM (label 1)
top_spam_words = get_top_words(df_train, label=1)
print("\nTop 10 SPAM words:\n", top_spam_words)


Top 10 HAM words:
 u            99
would        93
state        92
pm           89
president    84
percent      76
call         73
secretary    71
mr           70
time         70
Name: count, dtype: int64

Top 10 SPAM words:
 money          795
account        662
bank           606
fund           565
u              444
business       393
transaction    347
country        337
transfer       326
company        318
Name: count, dtype: int64


## Extra features

In [12]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
'''money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()'''

# Reconstruct DataFrames from earlier variables
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})

# Define extra feature patterns
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "£", "$"])
suspicious_words = "|".join([
    "free", "cheap", "sex", "money", "account", "bank",
    "fund", "transfer", "transaction", "win", "deposit", "password"
])

# Add extra features to train set
train_df["money_mark"] = train_df["text"].str.contains(money_symbol_list, case=False) * 1
train_df["suspicious_words"] = train_df["text"].str.contains(suspicious_words, case=False) * 1
train_df["text_len"] = train_df["text"].apply(len)

# Add extra features to test set
test_df["money_mark"] = test_df["text"].str.contains(money_symbol_list, case=False) * 1
test_df["suspicious_words"] = test_df["text"].str.contains(suspicious_words, case=False) * 1
test_df["text_len"] = test_df["text"].apply(len)

# Preview
train_df.head()


,text,label,money_mark,suspicious_words,text_len
29,regard mr nelson smithkindly reply private ema...,1,1,0,75
535,able reach oscar supposed send pdb receive,0,1,0,42
695,huma abedin bim checking pat work jack jake re...,0,1,0,79
557,announced monday cant today,0,1,0,27
836,bank africaagence san pedro bp san pedro cote ...,1,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit on training text and transform
X_train_bow = vectorizer.fit_transform(train_df['text'])

# Transform test text using the same vocabulary
X_test_bow = vectorizer.transform(test_df['text'])

# Check shapes
print("BoW train shape:", X_train_bow.shape)
print("BoW test shape:", X_test_bow.shape)



BoW train shape: (800, 16932)
BoW test shape: (200, 16932)


In [14]:
from scipy.sparse import hstack

# Select only the extra features
train_extra = train_df[['money_mark', 'suspicious_words', 'text_len']].values
test_extra = test_df[['money_mark', 'suspicious_words', 'text_len']].values

# Combine sparse BoW matrix with extra features
X_train_final = hstack([X_train_bow, train_extra])
X_test_final = hstack([X_test_bow, test_extra])

# Check final shapes
print("Final training matrix shape:", X_train_final.shape)
print("Final test matrix shape:", X_test_final.shape)

Final training matrix shape: (800, 16935)
Final test matrix shape: (200, 16935)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [15]:
# Your code

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform training text
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text'])

# Transform test text using same vocabulary
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'])

# Print the shape of the TF-IDF matrices
print("TF-IDF Train shape:", X_train_tfidf.shape)
print("TF-IDF Test shape:", X_test_tfidf.shape)


TF-IDF Train shape: (800, 16932)
TF-IDF Test shape: (200, 16932)


## And the Train a Classifier?

In [22]:
# Your code
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the model
model = LogisticRegression(max_iter=1000)

# Train on TF-IDF features
model.fit(X_train_tfidf, train_df['label'])

# Predict on test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print("Logistic Regression Report:\n")
print(classification_report(test_df['label'], y_pred))

# Optional: Confusion matrix
print("Confusion Matrix:\n")
print(confusion_matrix(test_df['label'], y_pred))


Logistic Regression Report:

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       125
           1       1.00      0.89      0.94        75

    accuracy                           0.96       200
   macro avg       0.97      0.95      0.96       200
weighted avg       0.96      0.96      0.96       200

Confusion Matrix:

[[125   0]
 [  8  67]]


In [18]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, train_df['label'])
y_pred_nb = nb_model.predict(X_test_tfidf)

# Evaluate the model
print("Naive Bayes Report:\n")
print(classification_report(test_df['label'], y_pred_nb))

# Optional: Confusion matrix
print("Confusion Matrix:\n")
print(confusion_matrix(test_df['label'], y_pred))


Naive Bayes Report:

              precision    recall  f1-score   support

           0       0.99      0.90      0.94       125
           1       0.85      0.99      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200

Confusion Matrix:

[[125   0]
 [  8  67]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [19]:
# Your code

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

nb = MultinomialNB()
nb.fit(X_train_tfidf, train_df['label'])
y_pred = nb.predict(X_test_tfidf)

# Evaluate the model
print("MultinomialNB Report:\n")
print(classification_report(test_df['label'], y_pred))

# Optional: Confusion matrix
print("Confusion Matrix:\n")
print(confusion_matrix(test_df['label'], y_pred))


MultinomialNB Report:

              precision    recall  f1-score   support

           0       0.99      0.90      0.94       125
           1       0.85      0.99      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200

Confusion Matrix:

[[112  13]
 [  1  74]]


### 🎯 Interpretation
Logistic Regression: more conservative — never mislabels ham, but misses some spam

MultinomialNB: more aggressive — catches nearly all spam, but more false alarms